In [1]:
import glob2
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
files = glob2.glob("train/*")
species = []
for f in files:
    species.append(f[6:]) # birds species list

Birds information data scrap from wikipedia - to be updated

In [3]:
%%script false --no-raise-error # dont execute cell

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re

webpage = r"https://en.wikipedia.org/wiki/Main_Page"

for searchterm in species[:4]:

    driver = webdriver.Chrome()
    driver.minimize_window()
    driver.get(webpage)

    # find search-box
    sbox = driver.find_element(By.CLASS_NAME, "vector-search-box-input")
    try:
        sbox.send_keys(searchterm) # write searching keywords in search-box
        submit = driver.find_element(By.ID, "searchButton")
        submit.click()

        try: # if searchterm correction is available - use it
            driver.find_element(By.XPATH, "//*[@class='mw-search-result'][1]/div[1]/a").click()
            found = "found with 'did you mean' correction"
        except NoSuchElementException:
            found = "found"
        
        # get basic information of specie
        basic_info = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/p[2]').text
        # clean text
        basic_info = re.sub('\s?\[.*?\]', '', basic_info).strip()
    
    except NoSuchElementException:
        found = "NOT found"

    print("searching for", searchterm, "-", found)

    try: # look for 'description' heading
        driver.find_element(By.ID, "Description")
        description = driver.find_element(By.XPATH, '//*[@id="Description"]/parent::h2/following-sibling::p').text
        description = re.sub('\s?\[.*?\]', '', description).strip()
    except NoSuchElementException:
        description = ""

    driver.close()

    print("information sample:", basic_info[:75])
    print("description sample:", description[:75])

Couldn't find program: 'false'


Definition to create array of bird species catalogues

In [4]:
def get_species_dataset(directory, species_count = None, img_count = None):
                                # reduced number of species and images per specie
    target = []

    zero = glob2.glob(os.path.join(directory, species[0])+"/*")[0]
    zero_img = cv2.cvtColor(cv2.imread(zero),cv2.COLOR_BGR2RGB).reshape(1,224,224,3)

    for specie in species[:species_count]:
        specie_id = species.index(specie)
        specie_imgs = glob2.glob(os.path.join(directory, specie)+"/*")

        for img in specie_imgs[:img_count]:
            tmp_img = cv2.cvtColor(cv2.imread(img),cv2.COLOR_BGR2RGB).reshape(1,224,224,3)
            
            if specie == species[0] and img == specie_imgs[0]:
                dataset = zero_img
                
            else:
                dataset = np.concatenate((dataset, tmp_img ), axis=0)

            target.append(specie_id)
            
    target_arr = np.array(target).reshape(-1,1)

    print("            dataset shape:", dataset.shape)
    print("             target shape:", target_arr.shape)
    print("target classes and counts:", dict((i, target.count(i)) for i in target))

    return dataset, target_arr

Dataset for DL training

In [5]:
x_train, y_train = get_species_dataset("train", 5)

            dataset shape: (808, 224, 224, 3)
             target shape: (808, 1)
target classes and counts: {0: 166, 1: 180, 2: 179, 3: 137, 4: 146}


In [6]:
x_train, y_train = get_species_dataset("test", 5)

            dataset shape: (25, 224, 224, 3)
             target shape: (25, 1)
target classes and counts: {0: 5, 1: 5, 2: 5, 3: 5, 4: 5}
